# Forecast Tracing

In addition to the completely flexible `callback` data storage system,
PassengerSim also includes the ability to "trace" certain details of the 
simulation.  Traces are more aggregate than other callbacks, as they will
follow the average values of various measures over many samples.  Unlike
other aggregate measures, they allow for the selection of a limited number of 
micro-level details, e.g. path forecasts for specific paths, or bid prices on
specific legs.  This contrasts with the database functionality, which can
(relatively) efficiently store this information for *all* paths or legs.
Tracing allows the analyst to probe a simulation for details of interest
without becoming bogged down in the recording and storage of massive amounts
of data that really isn't needed.

In [ ]:
import passengersim as pax

pax.versions()

Here, we'll run a quick demo using the "3MKT" example model.  We'll
give AL1 the 'P' RM system to make it interesting.

In [ ]:
cfg = pax.Config.from_yaml(pax.demo_network("3MKT"))

cfg.simulation_controls.num_samples = 400
cfg.simulation_controls.burn_samples = 50
cfg.simulation_controls.num_trials = 1
cfg.db = None
cfg.outputs.reports.clear()

cfg.carriers.AL1.rm_system = "P"

sim = pax.Simulation(cfg)

The path forecast tracing capabilities allows us to record detailed data about a subset of 
Simulation path forecasts, so we can review them after the simulation.

In [ ]:
from passengersim.tracers.forecasts import (
    PathForecastTracer,
    fig_path_forecast_dashboard,
)

tracer = PathForecastTracer(path_ids=[1, 9])
tracer.attach(sim)

In [ ]:
summary = sim.run()

All the usual summary data remains available for review and analysis.

In [ ]:
summary.fig_carrier_revenues()

In [ ]:
summary.fig_fare_class_mix()

## Path Forecast Dashboards

For the selected paths, we can review a dashboard that shows the forecast mean and std dev 
for each fare class from each DCP through departure, the mean forecast within each timeframe,
as well as history data on yieldable and (if recorded seperately) priceable sales, and average
closure rates at each DCP.

In [ ]:
fig_path_forecast_dashboard(summary, path_id=1)

In [ ]:
fig_path_forecast_dashboard(summary, path_id=9)

## Tracers in Callback Data

All the underlying data for these reports is stored in the summary's `callback_data` attribute, if you want to access 
it to parse or visualize it differently.

In [ ]:
summary.callback_data.selected_path_forecasts

## Relationship to Callbacks

Unlike other callback data, the tracers are not stored by sample day, as that would 
generally create an overwhelming amount of data to store, and we are typically not 
interested in that much detail.  If we are interested in grabbing and storing path forecast
data for individual sample days, we can still do that with the regular callback interface.

In [ ]:
sim1 = pax.Simulation(cfg)

In [ ]:
@sim1.begin_sample_callback
def grab_forecasts(sim):
    if sim.sim.sample not in [300, 375]:
        return
    return {f"path-{p}": sim.sim.paths.select(path_id=p).get_forecast_data() for p in [1, 9]}

In [ ]:
summary1 = sim1.run()

When run like this, we capture not the average path forecast over the simulation,
but rather the exact path forecast for the selected paths (1 and 9) at 
sample days 300 and 375.  The data is stored in the `callback_data.begin_sample`
attribute:

In [ ]:
summary1.callback_data.begin_sample

We can review the details of each specific forecast by accessing 
the dashboard visualization.

In [ ]:
summary1.callback_data.begin_sample[0]["path-1"].dashboard()

We can also access individual sub-tables of forecast data as pandas DataFrames, 
to manipulate or visualize as we like.

In [ ]:
summary1.callback_data.begin_sample[0]["path-1"].history_sold_yieldable

In [ ]:
summary1.callback_data.begin_sample[0]["path-1"].mean_in_timeframe